SENTIMENT ANALYSIS

Aqui se llevara a cabo el analisis de sentiemiento de las reviews, utilizando el modelo creado en el archivo NLP.ipynb

Importamos las librerias necesarias

In [1]:
import os
import re
import joblib
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')

Importamos la ruta actual del directorio, nos servira para cargar y guardar, los archivos

In [2]:
ruta_actual = os.getcwd()

Importamos el modelo creado en NPL, llamado NPL_SVC

In [3]:
ruta_modelo = os.path.join(ruta_actual, 'modelo', 'NPL_SVC.pkl')
modelo_svc = joblib.load(ruta_modelo)

Importamos el dataframe con el que vamos a trabajar

In [5]:
ruta_df = r'C:\PROYECTO-INDIVIDUAL\Data\parquet\reviews.parquet'
df_procesar = pd.read_parquet(ruta_df)

Empezaremos con normalizar los datos

In [6]:
# Importamos las stopwords y el filtro a utilizar
stopwords_ingles = nltk.corpus.stopwords.words('english')
stopwords_spanish = nltk.corpus.stopwords.words('spanish')
stopwords_russian = nltk.corpus.stopwords.words('russian')
stopwords_portuguese = nltk.corpus.stopwords.words('portuguese')
stopwords = stopwords_ingles + stopwords_spanish + stopwords_russian + stopwords_portuguese
filtrar = []
if True:
    filtrar.append('game')
    filtrar.append('play')
    filtrar.append('games')
    filtrar.append('playing')

# Instanciamos un wordnetlemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# Creamos un diccionario con las tags de las clases de palabras que usaremos al lematizar
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

# Creamos una lista de recomendaciones lematizadas
recomendaciones_lemm = []
for recomendacion in df_procesar.review:
    recomendacion = re.sub("[^a-zA-Z]"," ",str(recomendacion))
    recomendacion = recomendacion.lower()
    recomendacion = nltk.word_tokenize(recomendacion)
    recomendacion = [palabra for palabra in recomendacion if len(palabra)>2]
    recomendacion = [palabra for palabra in recomendacion if not palabra in stopwords]
    recomendacion = [wordnet_lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in recomendacion]
    recomendacion = " ".join(recomendacion)
    recomendaciones_lemm.append(recomendacion)

# Creamos una columna con las reviews lematizadas
df_procesar['review_lemm'] = recomendaciones_lemm

# Creamos una lista con los valores de las recomendaciones
lista_recomendaciones = list(df_procesar['review_lemm'].values)

In [7]:
# Asignamos el max_features
max_features = 3000

# Creamos el vectorizador
cou_vec = CountVectorizer(max_features=max_features, stop_words="english", ngram_range=(1,2))

# Creamos una matriz con la lista de recomendaciones de manera vectorizada
matriz_reviews = cou_vec.fit_transform(lista_recomendaciones)

# Transformamos en array la matriz
x = matriz_reviews.toarray()

# Hacer predicciones con los datos
predicciones = modelo_svc.predict(x)

Una vez predichos los resultados, asignamos esas predicciones a una nueva columna

In [8]:
# Guardamos en columna nueva
df_procesar['sentiment_analysis'] = predicciones

# Dropeamos columnas innecesarias
df_procesar.drop(columns=['review', 'review_lemm'], inplace=True)

Guardamos el df actualizado

In [9]:
ruta_guardar = r'C:\PROYECTO-INDIVIDUAL\Data\parquet\review_predicha.parquet'
df_procesar.to_parquet(ruta_guardar, index=False)